In [8]:
import numpy as np
import itk
import os
import nibabel as nib
from os import listdir, mkdir
from os.path import isdir, join

In [11]:
# Function to calculate TRE
def TRE(landmarks1, landmarks2, spacing):
    """
    mean (and standard deviation) 3D Euclidean magnitude distance 
    between calculated and reference landmark positions for the 
    set of validation landmarks. All values are reported in units of millimeters.
    
    Parameters:    
        landmarks1 (ndarry): first set of landmarks.
        landmarks2 (ndarry): second set of landmarks.
        spaing (tuple(float)): pixel spacing for x, y, z.
    
    Returns:
        (floar, float) mean and SD 3D Euclidean magnitude distance.
    """
    landmarks1 = spacing*landmarks1
    landmarks2 = spacing*landmarks2
    diff = landmarks1 - landmarks2
    squared = diff * diff
    dist = np.sqrt(np.sum(squared,axis=1))
    mean_TRE = np.mean(dist)
    sd_TRE = np.std(dist)

    return dist, mean_TRE, sd_TRE

#Import transformix text file and convert to np array
def transformix2np(path_transformix, no_points=300):
    import re
    landmarks = open(path_transformix, "r")
    reg_expr = r'OutputIndexFixed = \[([\d.\s\-]+)\]'
    landmarks_array = np.zeros((no_points, 3))

    for index, line in enumerate(landmarks):
        match_obj = re.search(reg_expr, line, re.M)
        coords = match_obj.group(1).split()
        coords = [round(float(c)) for c in coords]
        landmarks_array[index,:] = coords
    return landmarks_array

# Read nii from path and normalize array from 0 to 1
def read_im(image_path):
    nii_img = nib.load(image_path)
    nii_data = nii_img.get_data()
    return nii_data, nii_img

In [ ]:
# Set chosen image
chosen_im = 'copd1'
chosen_param = 'par0000'

# Set path to data folder containing copd1, copd2, etc. folders
data_dir = "../data"

#Set paths to landmarks and images
path_landmarkse = os.path.join(data_dir, f"{chosen_im}/{chosen_im}_300_eBH_xyz_r1.txt")
path_landmarksi = os.path.join(data_dir, f"{chosen_im}/{chosen_im}_300_iBH_xyz_r1.txt")
path_imge = os.path.join(data_dir, f'./{chosen_im}/{chosen_im}_eBHCT.nii.gz')
path_imgi = os.path.join(data_dir, f'./{chosen_im}/{chosen_im}_iBHCT.nii.gz')

# Set paths to output of transformix
path_transformix = f"../registration-results/{chosen_param}/{chosen_im}/outputpoints.txt"

In [ ]:
# Load landmark points and image
landmarkse = np.loadtxt(path_landmarkse)
landmarksi = np.loadtxt(path_landmarksi)

nii_data, nii_img = read_im(path_imge)

In [ ]:
dist, mean_TRE, SD_TRE = TRE(landmarkse, landmarksi, nii_img.header.get_zooms())
print(mean_TRE, SD_TRE)

In [ ]:
transformix_landmarks = transformix2np(path_transformix, no_points=300)
dist, mean_TRE, SD_TRE = TRE(landmarksi, transformix_landmarks, nii_img.header.get_zooms())
print(mean_TRE, SD_TRE)

## Section to Compute TRE for parameter file

In [24]:
# Compute TRE for four copd images for a chosen_parameter file
# Result is saved in folder "./registration-results/TRE-results"
chosen_param = 'par0035'

In [23]:
reg_dir = f"../registration-results/{chosen_param}"
data_dir = "../data"
onlydirs = [f for f in listdir(reg_dir) if isdir(join(reg_dir, f))]
# Array to save TRE results
TRE_results = np.zeros((len(onlydirs),3))
TRE_file_path = os.path.join("../registration-results/TRE-results",f"{chosen_param}_TRE_results.csv")

for idx, chosen_im in enumerate(onlydirs):
    # Load inhale image and get pixel spacing
    path_imge = os.path.join(data_dir, f'./{chosen_im}/{chosen_im}_iBHCT.nii.gz')
    inhale_img = nib.load(path_imge)
    # Load inhale image landmarks
    path_landmarksi = os.path.join(data_dir, f"{chosen_im}/{chosen_im}_300_iBH_xyz_r1.txt")
    landmarksi = np.loadtxt(path_landmarksi)
    # Load transformed parameters
    path_transform_param = f"../registration-results/{chosen_param}/{chosen_im}/outputpoints.txt"
    transformix_landmarks = transformix2np(path_transform_param, no_points=300)
    # Compute TRE
    _, mean_TRE, SD_TRE = TRE(landmarksi, transformix_landmarks, inhale_img.header.get_zooms())
    TRE_results[idx, :] = chosen_im[4], mean_TRE, SD_TRE

# Write results into folder "../registration-results/TRE-results"
with open(TRE_file_path, 'w+') as out_f:
    for index, row in enumerate(TRE_results): 
        out_f.write(','.join(str(j) for j in row) + '\n')

## Section to Write Elastix Commands to file

In [18]:
# Write commands for elastix test set registration
# Add parameter folder name and paramter file names (must be saved in ./parameter-files folder)
param_file = 'par0049' # folder name where results will be saved
param_1 = 'Par0035.SPREAD.MI.af.0.txt'
param_2 = 'Par0035.SPREAD.MI.bs.1.ASGD.txt'
data_dir = "../data"
test_ims = [f for f in listdir(data_dir) if isdir(join(data_dir, f))]
result_folder = f"../registration-results/{param_file}/"
mkdir(result_folder)

In [2]:
# Section writes to file in src folder
with open(f"elastix_transformix_{param_file}", 'w+') as out_f:
    
    for chosen_im in test_ims:
        fixed_im_path = os.path.join(data_dir, f'./{chosen_im}/{chosen_im}_iBHCT.nii.gz') 
        moving_im_path = os.path.join(data_dir, f'./{chosen_im}/{chosen_im}_eBHCT.nii.gz')
        result_path = f"../registration-results/{param_file}/{chosen_im}/"
        param_1_path = f"../parameter_files/{param_1}"
        param_2_path = f"../parameter_files/{param_2}"    
        mkdir(result_path)
        cmd = f"elastix -f {fixed_im_path} -m {moving_im_path} -out {result_path} -p {param_1_path} -p {param_2_path}\n"
        out_f.write(cmd)
        
        def_path = f"../data/{chosen_im}/{chosen_im}_300_iBH_xyz_r1_elastix.txt"
        result_path = f"../registration-results/{param_file}/{chosen_im}/"
        parameters_path = f"../registration-results/{param_file}/{chosen_im}/TransformParameters.1.txt"
        cmd = f"transformix -def {def_path} -out {result_path} -tp {parameters_path}\n"
        out_f.write(cmd)

In [19]:
# Write commands for transformix transformation
with open(f"test_transformation_{param_file}", 'w+') as out_f:
    
    for chosen_im in test_ims:
        def_path = f"../data/{chosen_im}/{chosen_im}_300_iBH_xyz_r1_elastix.txt"
        result_path = f"../registration-results/{param_file}/{chosen_im}/"
        parameters_path = f"../registration-results/{param_file}/{chosen_im}/TransformParameters.0.txt"
        cmd = f"transformix -def {def_path} -out {result_path} -tp {parameters_path}\n"
        out_f.write(cmd)